<a href="https://colab.research.google.com/github/MoosaSaadat/handsonml/blob/master/tensorflow_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print('TensorFlow Version:', tf.__version__)